In [1]:
import tensorflow as tf
from jobshop_env import Factory
from DQN_agent import DQN_agent_PER as DQN_agent
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")
from tqdm import tqdm

max_steps_per_episode = 1000
update_target_network = 5000
update_per_actions = 4
n_jobs = 6
n_machines = 6
channel = 5
n_states = (channel, n_jobs, n_machines)
n_actions = 9

env = Factory()
agent = DQN_agent(n_states, n_actions)

In [2]:
# tensorboardX
import time
from tensorboardX import SummaryWriter

In [3]:

total_episode = 30000
agent = DQN_agent(n_states, n_actions) #建立agent物件
env = Factory() #建立environment物件
for episode in tqdm(range(1, total_episode+1)): #每個訓練回合皆進行一次排程
    state = env.reset() #初始化模擬環境，推進到第一個學習事件
    episode_reward = 0
    for timestep in range(1, max_steps_per_episode):
        agent.frame_count += 1 #紀錄步數
        action = agent.choose_action(state) # 動作選擇
        agent.decay_epsilon()  # 探索率衰減

        next_state, reward, done, inf = env.step(action) #推進到下一個學習事件

        episode_reward += reward

        agent.store(state, action, next_state, reward, done) # 每次互動儲存訓練資料

        state = next_state

        if agent.frame_count % update_per_actions == 0 and len(agent.buffer.buffer) >= agent.batch_size:
            #每update_per_actions步進行神經網路學習
            agent.train_q_network()

        if agent.frame_count % update_target_network == 0:
            #每update_target_network步更新target神經網路學習
            agent.update_target_network()

        if done:
            break
    
agent.q_network.save('Model/ft06_DQN_model.h5')

100%|█████████▉| 29999/30000 [9:00:24<00:01,  1.08s/it]  

In [4]:
state = env.reset()
agent.epsilon = 0.0

while True:
    action = agent.choose_action(state)
    print("Time:"+str(env.env.now)+" Dispatching Rule:",action)
    next_state, reward, done, inf = env.step(action)
    
    state = next_state

    if done:
        print(inf)
        break

Time:0.0 Dispatching Rule: 6
Time:3.0 Dispatching Rule: 2
Time:9.0 Dispatching Rule: 5
Time:11.0 Dispatching Rule: 6
Time:14.0 Dispatching Rule: 2
Time:31.0 Dispatching Rule: 6
58.0
